In [2]:
import pickle
from glob import glob
import os
import yaml
from easydict import EasyDict as edict
import pandas as pd
import networkx as nx
import collections
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import torch
import torch.nn as nn
dirs = glob('../GNN_exp/msg_gnn/Msg*')
# dirs = glob('exp/node_gnn/Node*')
dirs.sort(key=os.path.abspath)

In [3]:
temp = pickle.load(open('../data_christmas/test_II/graph_barbell_nn100_0000001.p', 'rb'))
temp.keys()

dict_keys(['prob_gibbs', 'prob_hmc', 'J', 'b', 'seed_train', 'block_size', 'travel_time', 'time_gibbs', 'time_hmc', 'G', 'msg_node', 'idx_msg_edge', 'J_msg'])

In [4]:
temp['msg_node'].shape

(4902, 2)

In [5]:
temp['idx_msg_edge'].shape

(235396, 2)

In [6]:
temp['J_msg'].shape

(4902, 1)

In [20]:
from utils.topology import get_msg_graph
dirs = glob('../data_christmas/test_IV/*.p')
dirs.sort(key=os.path.abspath)

for ii, dr in enumerate(dirs):
    graph = pickle.load(open(dr, 'rb'))

    G = nx.from_numpy_array(graph['J'].todense())
    msg_node, msg_adj = get_msg_graph(G)
    msg_node, msg_adj = np.array(msg_node), np.array(msg_adj)
    idx_msg_edge = np.transpose(np.nonzero(msg_adj))
    J_msg = graph['J'].todense()[msg_node[:, 0], msg_node[:, 1]].reshape(-1, 1)

    graph['msg_node'] = msg_node
    graph['idx_msg_edge'] = idx_msg_edge
    graph['J_msg'] = J_msg

    with open(dr, 'wb') as f:
        pickle.dump(graph, f)
        del graph

In [17]:
temp['J'].shape

(9, 9)

In [12]:
temp0 = pickle.load(open('../data_christmas/train/graph_barbell_nn9_0000001.p', 'rb'))
temp0.keys()

dict_keys(['prob_gt', 'J', 'b', 'seed_train', 'G'])

In [13]:
temp0['J'].shape

(9, 9)

In [5]:
dirs = glob('../../Documents/train/*.p')
dirs.sort(key=os.path.abspath)

from scipy.sparse import coo_matrix
from utils.topology import get_msg_graph

idx_mat = np.random.permutation(len(dirs)).reshape(-1,10)

for jj, idx_row in enumerate(idx_mat):
    if jj % 10 == 0:
        print(jj)
        
    graph = {}
    batch = []
    for ii in idx_row:
        train_data = pickle.load(open(dirs[ii], 'rb'))
        batch.append(train_data)
    # ================================================
    graph['prob_gt'] = np.concatenate([bch['prob_gt'] for bch in batch], axis=0)
    graph['b'] = np.concatenate([bch['b'] for bch in batch], axis=0)
    graph['J_msg'] = np.concatenate([bch['J_msg'] for bch in batch], axis=0)

    n = graph['b'].shape[0]
    graph['J'] = coo_matrix(np.zeros([n, n]))
    
    row = []
    col = []
    val = []
    for ii, bch in enumerate(batch):
        nv = bch['J'].shape[0]
        row.append(bch['J'].row + nv * ii)
        col.append(bch['J'].col + nv * ii)
        val.append(bch['J'].data)

    graph['J'].row = np.concatenate(row)
    graph['J'].col = np.concatenate(col)
    graph['J'].data = np.concatenate(val)
        
    graph['seed_train'] = 11111
            
    # msg_node2 = msg_node1 + len(msg_node1)
    # msg_adj2 = msg_adj1 + msg_adj1.shape[0]
    # idx_msg_edge == np.vstack([idx_msg_edge1, msg_node1.shape[0] + idx_msg_edge2])

    G = nx.from_scipy_sparse_matrix(graph['J'])
    msg_node, msg_adj = get_msg_graph(G)
    msg_node = np.array(msg_node)
    msg_adj = np.array(msg_adj)
    idx_msg_edge = np.transpose(np.nonzero(msg_adj))
    
    tp_msg_node = np.empty((0,2))
    tp_idx_msg_edge = np.empty((0,2))
    num_msg_node = 0
    for bch in batch:
        tp_idx_msg_edge = np.vstack((tp_idx_msg_edge, tp_msg_node.shape[0] + bch['idx_msg_edge']))
        tp_msg_node = np.vstack((tp_msg_node, num_msg_node + bch['msg_node']))
        num_msg_node = 1 + tp_msg_node.max()
        
    print((msg_node==tp_msg_node).all())
    print((idx_msg_edge==tp_idx_msg_edge).all())
    import pdb
    pdb.set_trace()
    
    graph['msg_node'] = msg_node
    graph['msg_adj'] = idx_msg_edge
    
    graph['J'] = graph['J'].tolil()
    temp = [graph['J'][msg_node[r, 0], msg_node[r, 1]] for r in range(msg_node.shape[0])]
    graph['J'] = np.array(temp).reshape(-1,1)
    

#     file_name = os.path.join('../../Documents/', 'train', 'graph_nn9_{:07d}.p'.format(1+jj))
#     with open(file_name, 'wb') as f:
#         pickle.dump(graph, f)
#         del graph




0
True
True
> <ipython-input-5-a43c0b00a065>(64)<module>()
-> graph['msg_node'] = msg_node
(Pdb) q


BdbQuit: 

In [ ]:
# for idx in range(37):
#     test_type = 'test_II'
#     config_file = glob(dirs[idx] + '/*.yaml')[0]
#     config = yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader)
#     config['exp_dir'] = dirs[idx]
#     config['dataset']['data_path'] = '../data_christmas/'
#     config['dataset']['split'] = test_type
#     config['test']['test_model'] = dirs[idx] + '/model_snapshot_best.pth'
#     config['test']['batch_size'] = 1
#     cfg_path = dirs[idx] + '/config_test.yaml'
#     with open(cfg_path, 'w') as ymlfile:
#         yaml.dump(config, ymlfile, explicit_start=True)

#     ! python3 run_exp_local.py -c {cfg_path} -t

In [ ]:
df = pd.DataFrame(np.random.randn(0, 7), 
                  columns=["bvl", "hidden_dim", "num_prop", "aggregate_type", "loss", "lr", "wd"])
test_type = 'test_II'
bvl = []
for ii, dr in enumerate(dirs):
    train_files = glob(dr + '/*.p')
    train_data = pickle.load(open(train_files[0], 'rb'))
    
    test_files = glob(dr + '/*' + test_type + '.csv')
    q = pd.read_csv(test_files[0], sep='\t', header=None).values.reshape(13,1000,2)
    p = pd.read_csv(test_files[1], sep='\t', header=None).values.reshape(13,1000,2)

    config_file = glob(dr + '/*.yaml')[0]
    config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))
        
    vl = train_data['val_loss']
    bvl = train_data['best_val_loss'][0]
    arg_bvl = np.asarray(vl == bvl).nonzero()[0][0]

    df = df.append({"bvl":bvl,
                    "hidden_dim":config.model.hidden_dim,
                    "num_prop":config.model.num_prop,
                    "aggregate_type":config.model.aggregate_type,
                    "loss":config.model.loss,
                    "lr":config.train.lr,
                    "wd":config.train.wd,
                    "vl":vl,
                    "arg_bvl":arg_bvl,
                    "v100_p":p,
                    "v100_q":q}, ignore_index=True)

df = df.sort_values(['loss', 'bvl'], ascending=[True, True])
df_KLqp, df_MSE, df_KLpq = df[:9], df[9:17], df[17:27]
# df_KLpq, df_KLqp, df_MSE = df[:12], df[12:24], df[24:]
# df_KLqp, df_KLpq, df_MSE = df[:12], df[12:24], df[24:]
# df_MSE, df_KLqp, df_KLpq = df[:12], df[12:24], df[24:]

### Model performance on 20 validation graphs of |$\mathcal{V}$| = 9 and 13 fixed structures
#### Models with different hyperparameters, sorted by "KL-pq" Loss

In [ ]:
df_KLpq.drop(['vl', 'arg_bvl', 'v100_p', 'v100_q'], axis=1)

In [ ]:
num_model = len(df_KLpq)
f, axes = plt.subplots(nrows=int(np.ceil(num_model/6)), ncols=6, figsize=(25, 3*int(np.ceil(num_model/6))), dpi=100)
for ii in range(num_model):
    r = ii // 6
    c = ii % 6
    
    vl = df_KLpq.iloc[ii]['vl']
    bvl = df_KLpq.iloc[ii]['bvl']
    arg_bvl = df_KLpq.iloc[ii]['arg_bvl']
    
    axes[r,c].plot(vl, markerfacecolor="None", color='k', lw=0.3, label='val-loss')
    axes[r,c].plot(arg_bvl, bvl, '^', ms=7, mfc="None", color='r', label='best-val-loss')
    axes[r,c].set_title("ID {} : {:.2e}".format(df_KLpq.index[ii], bvl))
    axes[r,c].set_ylim([-1e-3, 0.01])

# plt.show()
plt.savefig('fig0.pdf', format='pdf')

### Model performance on 10 test graphs of |$\mathcal{V}$| = 9 and the same structure with training set

In [ ]:
Loss_test = np.zeros((4,num_model,13)) + np.nan

In [ ]:
topology = ['barbell', 'binarytree', 'bipartite', 'circladder', 'complete', 
            'cycle', 'grid', 'ladder', 'lollipop', 'path', 'star', 'tripartite', 'wheel']
loss_func = nn.KLDivLoss(reduction='batchmean')
f, axes = plt.subplots(nrows=num_model, ncols=13, figsize=(40, 3*num_model), dpi=100)
test_type = 'test_I'

for r in range(num_model):
    idx = df_KLpq.index[r]
    config_file = glob(dirs[idx] + '/*.yaml')[0]
    config = yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader)
    config['exp_dir'] = dirs[idx]
    config['dataset']['data_path'] = '../data_christmas/'
    config['dataset']['split'] = test_type
    config['test']['test_model'] = dirs[idx] + '/model_snapshot_best.pth'
    config['test']['batch_size'] = 1
    cfg_path = dirs[idx] + '/config_test.yaml'
    with open(cfg_path, 'w') as ymlfile:
        yaml.dump(config, ymlfile, explicit_start=True)

    ! python3 run_exp_local.py -c {cfg_path} -t
    
    test_files = glob(dirs[idx] + '/*' + test_type + '.csv')
    q = pd.read_csv(test_files[0], sep='\t', header=None).values.reshape(13,90,2)
    p = pd.read_csv(test_files[1], sep='\t', header=None).values.reshape(13,90,2)
    
    for c in range(len(topology)):
        axes[r,c].plot([[0,0],[1,1]],'k:')
        axes[r,c].plot(p[c][:,0], q[c][:,0], '.', markerfacecolor="None", color='b')
        axes[r,c].set_xlim([0, 1])
        axes[r,c].set_ylim([0, 1])
        axes[r,c].set_xticklabels([])
        #axes[r,c].set_yticklabels([])
        axes[r,c].axis('on')
        
        P = torch.Tensor(p[c])
        Q = torch.Tensor(q[c])
        
        loss = loss_func(torch.log(Q), P).item()
        Loss_test[0,r,c] = loss
        if r == 0:
            axes[r,c].set_title(topology[c]+"\n {:.2e}".format(loss))
        else:
            axes[r,c].set_title("\n {:.2e}".format(loss))
            
        if c == 0:
            axes[r,c].set_yticks([])
            axes[r,c].set_ylabel("ID {}".format(df_KLpq.index[r]))
        else:
            axes[r,c].set_yticklabels([])
            
# plt.show()
plt.savefig('fig1.pdf', format='pdf')

### Model performance on 10 test graphs of |$\mathcal{V}$| = 100 and the same structure with training set

In [ ]:
f, axes = plt.subplots(nrows=num_model, ncols=13, figsize=(40, 3*num_model), dpi=100)

for r in range(num_model):
    p = df_KLpq.iloc[r]['v100_p']
    q = df_KLpq.iloc[r]['v100_q']
    for c in range(len(topology)):
        if c == 8:
            axes[r,c].plot([[0,0],[1,1]],'k:')
            for ii in range(10):
                axes[r,c].plot(p[c][100*ii : 100*ii+51,0], q[c][100*ii : 100*ii+51,0], '.', markerfacecolor="None", color='b')
                axes[r,c].plot(p[c][100*ii+51 : 100*(ii+1):,0], q[c][100*ii+51 : 100*(ii+1),0], '.', markerfacecolor="None", color='r')
            axes[r,c].set_xlim([0, 1])
            axes[r,c].set_ylim([0, 1])
            axes[r,c].set_xticklabels([])
            #axes[r,c].set_yticklabels([])
            axes[r,c].axis('on')

            P = torch.Tensor(p[c])
            Q = torch.Tensor(q[c])

            loss = loss_func(torch.log(Q), P).item()
            Loss_test[1,r,c] = loss
            if r == 0:
                axes[r,c].set_title(topology[c]+"\n {:.2e}".format(loss))
            else:
                axes[r,c].set_title("\n {:.2e}".format(loss))

            if c == 0:
                axes[r,c].set_yticks([])
                axes[r,c].set_ylabel("ID {}".format(df_KLpq.index[r]))
            else:
                axes[r,c].set_yticklabels([])

plt.show()
# plt.savefig('fig2.pdf', format='pdf')

In [ ]:
f, axes = plt.subplots(nrows=num_model, ncols=13, figsize=(40, 3*num_model), dpi=100)

for r in range(num_model):
    p = df_KLpq.iloc[r]['v100_p']
    q = df_KLpq.iloc[r]['v100_q']
    for c in range(len(topology)):
        axes[r,c].plot([[0,0],[1,1]],'k:')
        axes[r,c].plot(p[c][:,0], q[c][:,0], '.', markerfacecolor="None", color='b')
        axes[r,c].set_xlim([0, 1])
        axes[r,c].set_ylim([0, 1])
        axes[r,c].set_xticklabels([])
        #axes[r,c].set_yticklabels([])
        axes[r,c].axis('on')
        
        P = torch.Tensor(p[c])
        Q = torch.Tensor(q[c])
        
        loss = loss_func(torch.log(Q), P).item()
        Loss_test[1,r,c] = loss
        if r == 0:
            axes[r,c].set_title(topology[c]+"\n {:.2e}".format(loss))
        else:
            axes[r,c].set_title("\n {:.2e}".format(loss))
            
        if c == 0:
            axes[r,c].set_yticks([])
            axes[r,c].set_ylabel("ID {}".format(df_KLpq.index[r]))
        else:
            axes[r,c].set_yticklabels([])
            
# plt.show()
plt.savefig('fig2.pdf', format='pdf')

### Model performance on 10 test graphs of the 
### Same Size ( |$\mathcal{V}$| = 9 ) & Same Degree Distribution but
### Anisomorphic with training set

In [ ]:
def degree(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())
    return degree_sequence, deg, cnt

In [ ]:
topology = ['barbell', 'binarytree', 'bipartite', 'circladder', 'complete', 
            'cycle', 'grid', 'ladder', 'lollipop', 'path', 'star', 'tripartite', 'wheel']
ignorable = ['binarytree', 'complete', 'cycle', 'path', 'star', 'tripartite', 'wheel']

f, axes = plt.subplots(nrows=10+num_model, ncols=13, figsize=(40, 3*(10+num_model)), dpi=100)
test_type = 'test_III'

for r in range(num_model):
    idx = df_KLpq.index[r]
    config_file = glob(dirs[idx] + '/*.yaml')[0]
    config = yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader)
    config['exp_dir'] = dirs[idx]
    config['dataset']['data_path'] = '../data_christmas/'
    config['dataset']['split'] = test_type
    config['test']['test_model'] = dirs[idx] + '/model_snapshot_best.pth'
    config['test']['batch_size'] = 1
    cfg_path = dirs[idx] + '/config_test.yaml'
    with open(cfg_path, 'w') as ymlfile:
        yaml.dump(config, ymlfile, explicit_start=True)

    ! python3 run_exp_local.py -c {cfg_path} -t
    
    test_files = glob(dirs[idx] + '/*' + test_type + '.csv')
    q = pd.read_csv(test_files[0], sep='\t', header=None).values.reshape(13-len(ignorable),90,2)
    p = pd.read_csv(test_files[1], sep='\t', header=None).values.reshape(13-len(ignorable),90,2)
    
    itp = 0
    for c, tp in enumerate(topology):
        if tp not in ignorable:
            if r == 0:
                data_files = glob('../data_christmas/' + test_type +'/*_' + tp + '_*.p')
                for idf, file in enumerate(data_files):
                    train_data = pickle.load(open(file, 'rb'))
                    
                    nx.draw(train_data['G'], ax=axes[idf, c])
                    _, dd, cc = degree(train_data['G'])
                    if idf == 0:
                        axes[idf, c].set_title(topology[c] + "\n d{}, c{}".format(dd, cc))
                    else:
                        axes[idf, c].set_title("d{}, c{}".format(dd, cc))
                
            axes[10+r,c].plot([[0,0],[1,1]],'k:')
            axes[10+r,c].plot(p[itp][:,0], q[itp][:,0], '.', markerfacecolor="None", color='b')
            axes[10+r,c].set_xlim([0, 1])
            axes[10+r,c].set_ylim([0, 1])
            axes[10+r,c].set_xticks([])
            axes[10+r,c].set_xticklabels([])
            #axes[10+r,c].set_yticklabels([])
            axes[10+r,c].axis('on')

            P = torch.Tensor(p[itp])
            Q = torch.Tensor(q[itp])
            
            loss = loss_func(torch.log(Q), P).item()
            Loss_test[2,r,c] = loss
            itp += 1

#             if r == 0:
#                 axes[10+r,c].set_title(topology[c]+"\n {:.2e}".format(loss))
#             else:
#                 axes[10+r,c].set_title("\n {:.2e}".format(loss))
            axes[10+r,c].set_title("\n {:.2e}".format(loss))

        if c == 0:
            axes[10+r,c].set_yticks([])
            axes[10+r,c].set_ylabel("ID {}".format(df_KLpq.index[r]))
        else:
            axes[r,c].set_xticks([])
            axes[r,c].set_xticklabels([])
            axes[r,c].set_yticks([])
            axes[r,c].set_yticklabels([])
            axes[10+r,c].set_xticks([])
            axes[10+r,c].set_xticklabels([])
            axes[10+r,c].set_yticks([])
            axes[10+r,c].set_yticklabels([])
            
# plt.show()
plt.savefig('fig3.pdf', format='pdf')

### Model performance on 10 test graphs of the 
### Same Size ( |$\mathcal{V}$| = 9 ) & Same Unique Degrees but
### Anisomorphic with training set

In [ ]:
topology = ['barbell', 'binarytree', 'bipartite', 'circladder', 'complete', 
            'cycle', 'grid', 'ladder', 'lollipop', 'path', 'star', 'tripartite', 'wheel']
ignorable = ['complete', 'cycle', 'path', 'star', 'tripartite', 'wheel']


f, axes = plt.subplots(nrows=10+num_model, ncols=13, figsize=(40, 3*(10+num_model)), dpi=100)
test_type = 'test_IV'

for r in range(num_model):
    idx = df_KLpq.index[r]
    config_file = glob(dirs[idx] + '/*.yaml')[0]
    config = yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader)
    config['exp_dir'] = dirs[idx]
    config['dataset']['data_path'] = '../data_christmas/'
    config['dataset']['split'] = test_type
    config['test']['test_model'] = dirs[idx] + '/model_snapshot_best.pth'
    config['test']['batch_size'] = 1
    cfg_path = dirs[idx] + '/config_test.yaml'
    with open(cfg_path, 'w') as ymlfile:
        yaml.dump(config, ymlfile, explicit_start=True)

    ! python3 run_exp_local.py -c {cfg_path} -t
    
    test_files = glob(dirs[idx] + '/*' + test_type + '.csv')
    q = pd.read_csv(test_files[0], sep='\t', header=None).values.reshape(13-len(ignorable),90,2)
    p = pd.read_csv(test_files[1], sep='\t', header=None).values.reshape(13-len(ignorable),90,2)
    
    itp = 0
    for c, tp in enumerate(topology):
        if tp not in ignorable:
            if r == 0:
                data_files = glob('../data_christmas/' + test_type +'/*_' + tp + '_*.p')
                for idf, file in enumerate(data_files):
                    train_data = pickle.load(open(file, 'rb'))
                    
                    nx.draw(train_data['G'], ax=axes[idf, c])
                    _, dd, cc = degree(train_data['G'])
                    if idf == 0:
                        axes[idf, c].set_title(topology[c] + "\n d{}, c{}".format(dd, cc))
                    else:
                        axes[idf, c].set_title("d{}, c{}".format(dd, cc))
                
            axes[10+r,c].plot([[0,0],[1,1]],'k:')
            axes[10+r,c].plot(p[itp][:,0], q[itp][:,0], '.', markerfacecolor="None", color='b')
            axes[10+r,c].set_xlim([0, 1])
            axes[10+r,c].set_ylim([0, 1])
            axes[10+r,c].set_xticks([])
            axes[10+r,c].set_xticklabels([])
            #axes[10+r,c].set_yticklabels([])
            axes[10+r,c].axis('on')

            P = torch.Tensor(p[itp])
            Q = torch.Tensor(q[itp])
            loss = loss_func(torch.log(Q), P).item()
            Loss_test[3,r,c] = loss
            itp += 1

#             if r == 0:
#                 axes[10+r,c].set_title(topology[c]+"\n {:.2e}".format(loss))
#             else:
#                 axes[10+r,c].set_title("\n {:.2e}".format(loss))
            axes[10+r,c].set_title("\n {:.2e}".format(loss))

        if c == 0:
            axes[10+r,c].set_yticks([])
            axes[10+r,c].set_ylabel("ID {}".format(df_KLpq.index[r]))
        else:
            axes[r,c].set_xticks([])
            axes[r,c].set_xticklabels([])
            axes[r,c].set_yticks([])
            axes[r,c].set_yticklabels([])
            axes[10+r,c].set_xticks([])
            axes[10+r,c].set_xticklabels([])
            axes[10+r,c].set_yticks([])
            axes[10+r,c].set_yticklabels([])
            
# plt.show()
plt.savefig('fig4.pdf', format='pdf')

In [ ]:
f, axes = plt.subplots(nrows=4, ncols=3, figsize=(20, 20), dpi=150)
for ii in range(num_model):
    r = ii // 3
    c = ii % 3
    axes[r,c].plot(np.log10(Loss_test[:,ii,:]).T, 'o-')
    if ii == 0:
        axes[r,c].legend(["|V|=009, structured","|V|=100, structured",
                          "|V|=009, random (same-degrees)","|V|=009, random (unique-degrees)"], 
                          bbox_to_anchor=(0.3, 1.2))
    if r == 3:
        axes[r,c].set_xticks(range(13))
        axes[r,c].set_xticklabels(topology, rotation=45, fontsize=12)
    else:
        axes[r,c].set_xticklabels([])
    axes[r,c].set_ylim([-6,0])
    axes[r,c].grid('true')
    axes[r,c].set_title("ID {} : Test Loss".format(df_KLpq.index[ii]))
    axes[r,c].set_ylabel('$\log_{10}(loss)$')

# plt.show()
plt.savefig('fig5.pdf', format='pdf')

In [ ]:
import pdb
pdb.set_trace()

In [ ]:
import collections
import networkx as nx
from utils.topology import NetworkTopology

def degree(G):
    degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
    degreeCount = collections.Counter(degree_sequence)
    deg, cnt = zip(*degreeCount.items())
    return degree_sequence, deg, cnt

def graph_to_adjacency_matrix(G):
    n = G.number_of_nodes()
    W = np.zeros([n, n])
    for i in range(n):
        W[i, [j for j in G.neighbors(i)]] = 1
    return W

In [ ]:
f, axes = plt.subplots(nrows=2, ncols=10, figsize=(40, 6), dpi=150)
graph9c_list = []
gfile = glob('*.g6')[0]
with open(gfile, 'rb') as file:  
    for ii, line in enumerate(file):
#         print(line)
        
#         if ii == 10:
#             break
            
        G = nx.from_graph6_bytes(line[:-1])
        graph9c_list.append(G)
#         W = graph_to_adjacency_matrix(G)
#         nx.draw(G, ax=axes[0,ii])
#         axes[1,ii].imshow(W)
        
with open('graph9c.p', 'wb') as f:
    pickle.dump(graph9c_list, f)

In [ ]:
graph9c_list = pickle.load(open('graph9c.p', 'rb'))

In [ ]:
len(graph9c_list)

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=13, figsize=(40, 3), dpi=150)
tp_list = ['barbell', 'binarytree', 'bipartite', 'circladder', 'complete', 
            'cycle', 'grid', 'ladder', 'lollipop', 'path', 'star', 'tripartite', 'wheel']
tp_list = ['lollipop']


ntp = NetworkTopology(num_nodes=9, seed=1234)
for ii, tt in enumerate(tp_list):
    G, _ = ntp.generate(topology=tt)
    
    if tt in ['bipartite', 'complete', 'tripartite']:
        nx.draw_circular(G, with_labels=True, ax=axes[ii])
    else:
        nx.draw(G, with_labels=True, ax=axes[ii])
        
    #mcb = nx.minimum_cycle_basis(G)
    #lc = np.unique([len(cy) for cy in mcb])
    #clq = nx.node_clique_number(G)
    #dm = nx.diameter(G)
    ccl = nx.cycle_basis(G)
    axes[ii].set_title("{}".format(ccl))
#     if tt == 'bipartite':
#         break

In [ ]:
clq

In [ ]:
lc

In [ ]:
# nx.draw_circular(G0, with_labels=True)
nx.draw(G0, with_labels=True)

In [ ]:
nx.minimum_cycle_basis(G0)

In [ ]:
nx.cycle_basis(G0)

In [ ]:
# G0 = nx.complete_multipartite_graph(3,3,3)
# G0 = nx.complete_bipartite_graph(4, 5)
# G0 = nx.cycle_graph(9)
# _, deg, count = degree(G0)
# W = graph_to_adjacency_matrix(G0)

# G0 = nx.grid_2d_graph(3,3)
# seq, deg, count = degree(G0)

# W = np.zeros([9,9])
# nodes = [n for n in G0.nodes]
# for i in range(9):
#     for j in G0.neighbors(nodes[i]):
#         W[i, 3 * j[0] + j[1]] = 1
    
# G = [G0]
# f, axes = plt.subplots(nrows=2, ncols=10, figsize=(40, 6), dpi=200)
# nx.draw(G0, ax=axes[0,0])
# axes[0,0].set_title("{}, {}".format(deg, count))
# axes[1,0].imshow(W)
    
# cnt = 0
# for i in range(1,100):
#     legitimate = True
#     try:
#         seq = unique_deg_preserved_seq(deg, count)
#         G_new = nx.random_degree_sequence_graph(seq)
#         for j in range(len(G)):
#             if nx.is_isomorphic(G[j],G_new) or not nx.is_connected(G_new):
#                 legitimate = False
#                 break
#         if legitimate:
#             G.append(G_new)
#             cnt += 1

#             nx.draw(G_new, ax=axes[0,cnt])
#             _, dd, cc = degree(G_new)
#             axes[0,cnt].set_title("{}, {}".format(dd, cc))
#             W = graph_to_adjacency_matrix(G_new)
#             axes[1,cnt].imshow(W)
#     except:
#         pass
    
#     if cnt == 9:
#         break